In [1]:
import os
import pickle
from apiclient.discovery import build
from ipynb.fs.defs.get_videos import get_channel_videos
import pickle
api_key = os.environ.get("YOUTUBE_API_KEY")
api_key2 = os.environ.get("YOUTUBE_API_KEY_2")
youtube = build('youtube', 'v3', developerKey=api_key2)

### Display keys for a dict

In [2]:
def display_keys(video, dict_key):
    item = get_item_from_video(video, dict_key)
    print(item.keys())

## Get all videos from a YouTube channel

In [3]:
# input 0 to get data from api
# input 1 to get data from file
get_from_file = 1
if get_from_file == 0:
    videos = get_channel_videos('UCyps-v4WNjWDnYRKmZ4BUGw', is_id=1)
else:
    original = pickle.load(open("original_videos.pickle", "rb")) 
    videos_detail = pickle.load(open("videos.pickle", "rb"))

## Helper functions to retrieve data from each video

In [4]:
def get_item_from_video(video, dict_key = "", original=0, snippet_item = "", statistics_item = ""):
    snippet = video["snippet"]
    video_title = snippet["title"]
    
    kind = video["kind"]
    etag = video["etag"]
    
    if(original == 1):
        video_id = snippet["resourceId"]["videoId"]
    else:
        # items only available from video details array
        video_id = video["id"]
        contentDetails = video['contentDetails']
        statistics = video['statistics']
    
    ## check what item is being asked for
    if(dict_key == "id"):
        return video_id
    elif(dict_key == "title"):
        return video_title
    elif(dict_key == "snippet"):
        return snippet
    elif(dict_key == "contentDetails"):
        return contentDetails
    elif(dict_key == "statistics"):
        return statistics
    elif(dict_key == "kind"):
        return kind
    elif(dict_key == "etag"):
        return etag
    elif(dict_key == ""):
        # used when we want to display the whole array
        # like in display_keys to get all main keys
        return video
    else:
        return 0

In [6]:
## filter videos based on title [Machine learning, AI, Deep Learning ....]

# def filter(videos)
# ....

## get much more details about each video

In [ ]:
def helper_get_single_video_details(video_id):
    res = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        id=video_id).execute()
    return res

def get_videos_details(videos):
    videos_detailed = []
    for video in videos:
        video_id = get_item_from_video(video, 'id', original=1)
        res = helper_get_single_video_details(video_id)
        videos_detailed += res['items']
    return videos_detailed

In [ ]:
# only need to retrieve video_details if this is from API and not from file
# since the file data has already done this step from API
if get_from_file == 0:
    videos_detail = get_videos_details(videos)

### Random testing

In [ ]:
dicti = {}
def working_on_getting_details(videos):
    for item in range(0,2):
        video = videos_detail[item]
        video_id = get_item_from_video(video, 'id')
        snippet = get_item_from_video(video, 'snippet')
        statistics = get_item_from_video(video, 'statistics')
        #displayKeys
        display_keys(video, "statistics")
        # results
        print(statistics["likeCount"])
        print(video_id)
        print(snippet)
        print(statistics)
        print("\n")
working_on_getting_details(videos_detail)

### Export to pickle file to be able to retrieve data later without having to run the entire thing again

In [ ]:
#import videos to file if the data is from API
if get_from_file == 0:
    pickle.dump(videos, open("original_videos.pickle","wb"))

pickle.dump(videos_detail, open("videos.pickle","wb"))